In [146]:
from keycloak import KeycloakAdmin
import pandas as pd

In [147]:
#!pip install python-keycloak

In [345]:
KEYCLOAK_URL = "http://localhost:8080/auth/"
USERNAME = "admin"
PASSWORD = 'Pa55w0rd'
MASTER_REALM_NAME = "master"
REALM_NAME = "enki"

In [357]:
keycloak_admin = KeycloakAdmin(server_url=KEYCLOAK_URL,
                               username=USERNAME,
                           password=PASSWORD,
                               verify=True)
keycloak_admin.realm_name = REALM_NAME

In [358]:
def create_role(admin, name, description):
    return admin.create_realm_role({'name': name, 'description': description, 'clientRole': True}, skip_exists=True)

def create_roles(admin):
    roles = pd.read_csv("../docker/scripts/roles.csv")
    for r, row in roles.where(pd.notnull(roles), None).iterrows():
        create_role(admin, row['name'], row['description'])
        
create_roles(keycloak_admin)

In [359]:
realm_roles = keycloak_admin.get_realm_roles()

In [360]:
def create_group(admin, name, parent):
    if parent:
        found_group = keycloak_admin.get_group_by_path(f"/{parent}")
        group = admin.create_group({"name":name}, parent=found_group["id"], skip_exists=True, )

    else:
        group = admin.create_group({"name":name}, skip_exists=True, )
    return group 

def assign_role_to_group(admin, group_id, roles, realm_roles):
    roles_ids = [realm_role["id"] for realm_role in realm_roles if realm_role["name"]in roles ]
    print(group_id)
    print(roles_ids)
    print([{"id": role_id} for role_id in roles_ids])
    return admin.assign_group_realm_roles(group_id=group_id, roles=[{"id": role_id} for role_id in roles_ids])
    
def create_groups(admin, realm_roles):
    groups = pd.read_csv("../docker/scripts/groups.csv")
    for r, row in groups.where(pd.notnull(groups), None).iterrows():
        _ = create_group(admin, row["name"], row.parent)
        found_group = admin.get_group_by_path(f"/{row['name']}")
        roles = row["roles"].split("/")
        #assign_role_to_group(admin, found_group["id"], roles=roles, realm_roles=realm_roles)
        
create_groups(keycloak_admin, realm_roles=realm_roles)

In [ ]:
def create_user(admin, username, password, code_commune):
    new_user = admin.create_user({"username": "rcourivaud",
                                       "enabled": True,
                                           "attributes":{"code_commune":code_commune},
                                       "credentials": [{"value":password, "type": "password"}],
                                       })
    return new_user
def assign_user_group(admin, user_id, group_id):
    return admin.group_user_add(user_id, group_id)


def create_users(admin):
    users = pd.read_csv("../docker/scripts/users_seed.csv")
    for r, row in users.where(pd.notnull(users), None).iterrows():
        user = create_user(admin, row['username'], row['password'], row['code_insee'])
        found_group = admin.get_group_by_path(f"/{row['group']}")
        assign_user_group(admin, user, found_group["id"])
                                              
create_users(keycloak_admin)

In [362]:
def create_user(admin, username, password, code_commune):
    new_user = admin.create_user({"username": "rcourivaud",
                                       "enabled": True,
                                           "attributes":{"code_commune":code_commune},
                                       "credentials": [{"value":password, "type": "password"}],
                                       })
def create_users(admin):
    users = pd.read_csv("../docker/scripts/users_seed.csv")
    for r, row in users.where(pd.notnull(users), None).iterrows():
        create_user(admin, row['username'], row['password'], row['code_insee'])
        

SyntaxError: invalid syntax (<ipython-input-362-27cdfa27fa44>, line 5)

In [364]:
create_users(keycloak_admin)

KeycloakAuthenticationError: 401: b'{"error":"HTTP 401 Unauthorized"}'